In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import torch
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple
import gensim.downloader
from torch.optim import Adam
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import pickle
from torch import nn

In [2]:
!git clone https://github.com/hrobjarturh/nlptasks.git

fatal: destination path 'nlptasks' already exists and is not an empty directory.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd nlptasks

/content/nlptasks


In [4]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import numpy as np
#from datasets import load_dataset
from transformers import AutoTokenizer
import math 
import torch

device = torch.device("cpu")
if torch.cuda.is_available():
 device = torch.device("cuda")
device

device(type='cuda')

In [6]:
#Load data
dataset = load_dataset("copenlu/answerable_tydiqa")
dataset

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 13325
    })
    train: Dataset({
        features: ['question_text', 'document_title', 'language', 'annotations', 'document_plaintext', 'document_url'],
        num_rows: 116067
    })
})

In [7]:
#split data 
train_set = dataset["train"]
validation_set = dataset["validation"]

train_set = train_set.to_pandas()
validation_set = validation_set.to_pandas()
validation_set

,question_text,document_title,language,annotations,document_plaintext,document_url
0,อยุธยามีกี่อำเภอ ?,จังหวัดพระนครศรีอยุธยา,thai,"{'answer_start': [42], 'answer_text': ['16']}",\nปัจจุบันจังหวัดพระนครศรีอยุธยาประกอบด้วย 16 ...,https://th.wikipedia.org/wiki/%E0%B8%88%E0%B8%...
1,كم عدد مرات فوز الأوروغواي ببطولة كاس العالم ل...,قائمة نهائيات كأس العالم,arabic,"{'answer_start': [394], 'answer_text': ['بطولت...",أقيمت البطولة 21 مرة، شارك في النهائيات 78 دول...,https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D8%...
2,Siapakah yang menemuka benua Amerika ?,Kristoforus Kolumbus,indonesian,"{'answer_start': [193], 'answer_text': ['orang...",Kolumbus bukanlah orang pertama yang tiba di A...,https://id.wikipedia.org/wiki/Kristoforus%20Ko...
3,化学兵器禁止条約はどこで採択された？,化学兵器禁止条約,japanese,"{'answer_start': [11], 'answer_text': ['パリ']}",1993年1月13日にパリにおいて署名がなされ、1997年4月29日に発効した[1]。実効的...,https://ja.wikipedia.org/wiki/%E5%8C%96%E5%AD%...
4,ใครเป็นผู้ก่อการ ใน เหตุโจมตีในนอร์เวย์ พ.ศ. 2...,เหตุโจมตีในนอร์เวย์ พ.ศ. 2554,thai,"{'answer_start': [400], 'answer_text': ['แอนเด...",เหตุโจมตีครั้งที่สองนั้นเกิดขึ้นในราวสองชั่วโม...,https://th.wikipedia.org/wiki/%E0%B9%80%E0%B8%...
...,...,...,...,...,...,...
13320,월드컵이 처음 개최된 나라는 어디인가?,FIFA 월드컵,korean,"{'answer_start': [-1], 'answer_text': ['']}",브라질의 펠레는 FIFA 월드컵에 출전한 모든 선수들 중 최고로 월드컵 기록이 좋은...,https://ko.wikipedia.org/wiki/FIFA%20%EC%9B%94...
13321,Berapakah berat Ikan pari manta yag terbesar?,Ikan pari manta,indonesian,"{'answer_start': [-1], 'answer_text': ['']}","Ciri khas manta adalah sepasang ""tanduk"" di de...",https://id.wikipedia.org/wiki/Ikan%20pari%20manta
13322,Когда впервые начали применять компьютерную гр...,CGI (графика),russian,"{'answer_start': [-1], 'answer_text': ['']}",Созданием движущихся изображений занимается ко...,https://ru.wikipedia.org/wiki/CGI%20%28%D0%B3%...
13323,หม่อมราชวงศ์สุขุมพันธุ์ บริพัตร เกิดเมื่อไหร่?,หม่อมราชวงศ์สุขุมพันธุ์ บริพัตร,thai,"{'answer_start': [-1], 'answer_text': ['']}",ลำดับหม่อมราชวงศ์สุขุมพันธุ์ บริพัตร\n\nพระบาท...,https://th.wikipedia.org/wiki/%E0%B8%AB%E0%B8%...


In [8]:
chosen_language = ['english', 'japanese', 'finnish']

train_set = train_set[train_set['language'].isin(chosen_language)]
validation_set = validation_set[validation_set['language'].isin(chosen_language)]

validation_set

,question_text,document_title,language,annotations,document_plaintext,document_url
3,化学兵器禁止条約はどこで採択された？,化学兵器禁止条約,japanese,"{'answer_start': [11], 'answer_text': ['パリ']}",1993年1月13日にパリにおいて署名がなされ、1997年4月29日に発効した[1]。実効的...,https://ja.wikipedia.org/wiki/%E5%8C%96%E5%AD%...
9,オリヴィア・デ・ハヴィランドが生まれたのはいつ,オリヴィア・デ・ハヴィランド,japanese,"{'answer_start': [46], 'answer_text': ['1916年7...","\nオリヴィア・デ・ハヴィランド（Dame Olivia De Havilland, DBE...",https://ja.wikipedia.org/wiki/%E3%82%AA%E3%83%...
11,Kauanko lasia on valmistettu?,Lasi,finnish,"{'answer_start': [160], 'answer_text': ['noin ...",Vanhin tunnettu lasilaatu on alkali­kalkki­las...,https://fi.wikipedia.org/wiki/Lasi
14,Mikä on Ponzi-huijaus?,Ponzi-huijaus,finnish,"{'answer_start': [39], 'answer_text': ['pyrami...",Ponzi-huijaus eli Ponzi-järjestelmä on pyramid...,https://fi.wikipedia.org/wiki/Ponzi-huijaus
20,Mikä oli Napoleonin sotien lopputulos?,Napoleonin sodat,finnish,"{'answer_start': [15], 'answer_text': ['Ranska...",Sotien jälkeen Ranska alistettiin kovilla rauh...,https://fi.wikipedia.org/wiki/Napoleonin%20sodat
...,...,...,...,...,...,...
13305,What is the most common first word by babies?,Vocabulary development,english,"{'answer_start': [-1], 'answer_text': ['']}","Social pragmatic theories, also in contrast to...",https://en.wikipedia.org/wiki/Vocabulary%20dev...
13307,Kuinka kauan valtiopäivät kestää?,Suomen valtiopäivät,finnish,"{'answer_start': [-1], 'answer_text': ['']}",\nEnnen vuotta 1906 kokoontuivat säätyvaltiopä...,https://fi.wikipedia.org/wiki/Suomen%20valtiop...
13315,アイスランド共和国の首都はどこですか？,アイスランド,japanese,"{'answer_start': [-1], 'answer_text': ['']}",こうした危機を乗り切るため、アイスランド中央銀行は8日にロシアから40億ユーロの緊急融資を受...,https://ja.wikipedia.org/wiki/%E3%82%A2%E3%82%...
13317,Milloin käytiin Persianlahden sota?,Persianlahden sota,finnish,"{'answer_start': [-1], 'answer_text': ['']}",Ilmaiskuilla oli erityisen tuhoisa vaikutus Ir...,https://fi.wikipedia.org/wiki/Persianlahden%20...


In [9]:
def get_answerability(annotations):
    answerable = []
    for annot in annotations:
        if -1 in annot['answer_start']:
            answerable.append(0)
        else:
            answerable.append(1)
    return answerable

train_set['answerable'] = get_answerability(train_set['annotations'].tolist())
validation_set['answerable'] = get_answerability(validation_set['annotations'].tolist())

train_set.columns

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Index(['question_text', 'document_title', 'language', 'annotations',
       'document_plaintext', 'document_url', 'answerable'],
      dtype='object')

In [10]:
from datasets import DatasetDict, Dataset

def question_parag_combine(questions, paragraphs):
    """
    This function combines the questions and paragraphs into a single text
    Args:
        questions: list of questions
        paragraphs: list of paragraphs
    Returns:
        list of combined questions and paragraphs
    """
    training_data = []
    for index in range(len(questions)):
        training_data += [questions[index] + "\n" + paragraphs[index]]
        
    return training_data

def make_df(train_set, validation_set):

  d_q = train_set['question_text'].tolist()
  d_p = train_set['document_plaintext'].tolist()
  training_data = question_parag_combine(d_q,d_p)
  training_labels = train_set['answerable'].tolist()

  d_q = validation_set['question_text'].tolist()
  d_p = validation_set['document_plaintext'].tolist()
  validation_data = question_parag_combine(d_q,d_p)
  validation_labels = validation_set['answerable'].tolist()



  data_set = {}
  sets = [['train',training_data, training_labels], ['val', validation_data, validation_labels]]
  for meta in sets:
      data_set[meta[0]] = {}
      data_set[meta[0]]['text'] = []
      data_set[meta[0]]['labels'] = []
      
      for ind, text in enumerate(meta[1]):
          data_set[meta[0]]['text'].append(text)
          data_set[meta[0]]['labels'].append(meta[2][ind])

          

          
  data_set = DatasetDict({'train':Dataset.from_dict(data_set['train']),
                          'valid':Dataset.from_dict(data_set['val'])\
                        })
  return data_set

data_set = make_df(train_set, validation_set)
data_set

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 29868
    })
    valid: Dataset({
        features: ['text', 'labels'],
        num_rows: 3712
    })
})

In [6]:
from transformers import XLMRobertaTokenizer, XLMRobertaForCausalLM, XLMRobertaConfig, XLMTokenizer, XLMConfig, XLMModel

tokenizer = XLMRobertaTokenizer.from_pretrained("roberta-base")
#config = XLMRobertaConfig.from_pretrained("xlm-roberta-base")
#config.is_decoder = True
#model = XLMRobertaForCausalLM.from_pretrained("roberta-base")

# Initializing a XLM configuration
#configuration = XLMConfig(causal = True)


OSError: ignored

In [7]:
from transformers import XLMRobertaTokenizer, XLMRobertaForCausalLM, XLMRobertaConfig
import torch

tokenizer = XLMRobertaTokenizer.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

#model = XLMRobertaForCausalLM.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
#outputs = model(**inputs)

#prediction_logits = outputs.logits

In [13]:
def question_parag_combine(questions, paragraphs):
    """
    This function combines the questions and paragraphs into a single text
    Args:
        questions: list of questions
        paragraphs: list of paragraphs
    Returns:
        list of combined questions and paragraphs
    """
    training_data = []
    for index in range(len(questions)):
        training_data += [questions[index] + "\n" + paragraphs[index]]
        
    return training_data

In [14]:
block_size = 128

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def group_texts(examples):
    # Concatenate all texts.
    keys = ['attention_mask', 'input_ids']
    concatenated_examples = {k: sum(examples[k], []) for k in keys}
    total_length = len(concatenated_examples[list(keys)[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    # this is needed as the used dataset is a subclass of ClassificationDataset, which requires label as a field...
    result["label"] = result["input_ids"].copy()
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_datasets = data_set.map(tokenize_function, batched=True, batch_size=1000, num_proc=4, remove_columns=["text"])
tokenized_datasets_lm = tokenized_datasets.map(group_texts, batched=True, batch_size=1000, num_proc=4)
tokenized_datasets_lm = tokenized_datasets_lm.remove_columns(['label'])

#0:   0%|          | 0/8 [00:00<?, ?ba/s]

#1:   0%|          | 0/8 [00:00<?, ?ba/s]

#2:   0%|          | 0/8 [00:00<?, ?ba/s]

#3:   0%|          | 0/8 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/8 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [5]:
import datasets

#tokenized_datasets_lm.save_to_disk('tokenized_dataset')


In [9]:
%pwd

'/content/nlptasks/data'

In [10]:
data_set = datasets.load_from_disk('tokenized_dataset')

In [11]:
from transformers import Trainer, TrainingArguments
#training
#model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    per_device_train_batch_size = 2
    #max_steps=300
)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_set['train'].shuffle(seed=42).select(range(5000)),
    eval_dataset= data_set['valid'],
)

In [16]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  Number of trainable parameters = 560142482


Epoch,Training Loss,Validation Loss
1,0.348900,0.238289


Saving model checkpoint to test-clm/checkpoint-500
Configuration saved in test-clm/checkpoint-500/config.json
Model weights saved in test-clm/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1000
Configuration saved in test-clm/checkpoint-1000/config.json
Model weights saved in test-clm/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1500
Configuration saved in test-clm/checkpoint-1500/config.json
Model weights saved in test-clm/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-2000
Configuration saved in test-clm/checkpoint-2000/config.json
Model weights saved in test-clm/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-2500
Configuration saved in test-clm/checkpoint-2500/config.json
Model weights saved in test-clm/checkpoint-2500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 14848
  Batch size = 8


Training completed. Do not forget to sha

TrainOutput(global_step=2500, training_loss=1.079426025390625, metrics={'train_runtime': 2280.8553, 'train_samples_per_second': 2.192, 'train_steps_per_second': 1.096, 'total_flos': 1165874204160000.0, 'train_loss': 1.079426025390625, 'epoch': 1.0})

In [ ]:
eval_results = trainer.evaluate()


print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


In [ ]:
eng_eval_results = eng_trainer.evaluate()
print(f"Perplexity: {math.exp(eng_eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1107
  Batch size = 8


Perplexity: 48.74


In [18]:
torch.save(model, '../models/multi_model.pth')

In [8]:
model = torch.load('models/multi_model.pth',map_location ='cpu')

In [9]:
# Export contextual representations:

def mean_pooling(model_output, attention_mask):
    # Mean Pooling - Take attention mask into account for correct averaging
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(model_output.size()).float()
    sum_embeddings = torch.sum(model_output * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


In [20]:
current_language = 'eng'
# import training data
dft = pd.read_csv('data/dft_'+current_language+'.csv')

# import validation data
dfv = pd.read_csv('data/dfv_'+current_language+'.csv')

In [21]:
seed = 5555
size_train = 5000
size_valid = 2000

In [22]:
arange_dft = np.arange(len(dft))
np.random.seed(seed)
np.random.shuffle(arange_dft)

dft = dft.iloc[arange_dft]

train_df = dft.iloc[:size_train]
#valid_df = dft.iloc[size_train:size_train+size_valid]
test_df = dfv

train_meta = train_df[['word_frequency_score','logres_pred']].values
#valid_meta = valid_df[['word_frequency_score','logres_pred']].values
test_meta = test_df[['word_frequency_score','logres_pred']].values

In [15]:
def create_batch(df, meta, model):
  combined = []
  que = df["question_text"].values
  text = df["document_plaintext"].values
  for index in range(len(df)):
    combined.append(str(que[index])+" "+str(text[index]))
  
  last_hidden = []
  for row in combined:
    encoded = tokenizer(combined[0],
                        return_tensors='pt',truncation=True,max_length=64)
    encoded = {k:v.to(device) for k, v in encoded.items()}
    with torch.no_grad():
      model_output = model(**encoded, output_hidden_states=True, return_dict=True)
    pooled_output  = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])
    pooled_output = pooled_output.detach().to(device)


    
    last_hidden.append(pooled_output)

  text_lens = [len(i) for i in combined] # length of all doc lists

  return combined, text_lens, df.label.values, meta
  

In [25]:
#train_dataset = create_batch(train_df, train_meta, model.to(device))
test_dataset = create_batch(test_df.iloc[:100],test_meta[:100], model.to(device)) # rest of training for testing

In [26]:
torch.save(test_dataset, 'data/multi_test_batch')

In [43]:
train_dataset[3]

array([[0.69230769, 1.        ],
       [0.6       , 1.        ],
       [0.4       , 1.        ],
       ...,
       [0.16666667, 0.        ],
       [0.45454545, 0.        ],
       [0.44444444, 1.        ]])

In [ ]:
combined = []
que = dfv["question_text"].values
text = dfv["document_plaintext"].values
for index in range(len(dfv)):
  combined.append(str(que[index])+" "+str(text[index]))

In [ ]:
train_data = []
valid_data = []
test_data = []
for row in combined:
  encoded = eng_tokenizer(combined[0],
                      return_tensors='pt')
  encoded = {k:v.to(device) for k, v in encoded.items()}

  model_output = eng_model(**encoded, output_hidden_states=True, return_dict=True)

  mean_model_output = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])

In [ ]:
  encoded = eng_tokenizer('This is words',
                      return_tensors='pt')
  encoded = {k:v.to(device) for k, v in encoded.items()}

  model_output = model(**encoded, output_hidden_states=True, return_dict=True)

  mean_model_output = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])

In [ ]:
mean_model_output

tensor([[-7.6618e-02,  3.0584e-01, -7.7772e-01,  2.7464e-01, -1.3996e-01,
          1.3528e-01,  2.2021e+00,  5.5615e-01, -1.6915e-01, -2.3814e-02,
          1.4020e-02,  4.3055e-02, -1.5713e-01,  1.1371e-01,  1.3940e-01,
          6.8808e-02, -2.5337e-01, -2.2795e-01,  3.8142e-02,  1.0883e-01,
          1.0344e-01, -1.0245e-01,  2.9589e-01, -3.3583e-01, -1.4743e-01,
         -9.0522e-02, -2.9030e-01, -2.0552e-01,  1.0589e-02,  1.2487e-01,
          2.2613e-02, -5.8727e-02, -7.3387e-02, -8.8789e-02, -6.8163e-03,
          3.8037e-01,  3.2271e+01,  2.7284e-01, -5.3693e-02,  5.0226e-01,
         -3.0682e-02,  2.8502e-01,  2.3457e-02,  2.2691e-02, -3.2705e-01,
         -8.4868e-02, -1.0843e-01, -2.5657e-01, -4.2096e-02,  5.6631e-01,
         -1.4963e-01,  3.5003e-01, -1.1763e-01, -5.4871e-03, -1.1295e-01,
          5.0442e-01, -5.6427e-04, -2.7593e-01,  5.1326e-02, -1.5914e-01,
          3.8852e-02,  1.5863e-01,  1.3620e-01, -1.2790e-01, -1.2219e+00,
         -8.4525e-02, -9.8851e-02, -2.

In [ ]:
print(model_output.keys())
print(len(model_output['hidden_states'])) # contextual representations of separate words from each of the 6 layes
print(model_output['hidden_states'][-1].shape) # last layer with contextual representations (batch_size x num words x representation dim)

# Aggregate all the representations into one
mean_model_output2 = mean_pooling(model_output['hidden_states'][-1], encoded['attention_mask'])


odict_keys(['logits', 'past_key_values', 'hidden_states'])
7
torch.Size([1, 68, 768])


In [ ]:
eng_tokenizer(eng_set['train']['text'][100],
                    return_tensors='pt')

{'input_ids': tensor([[ 2061,   389,   262,  9539,   286, 18522,    30,   198,   464,   509,
          9116, 43400,    12, 38328,  2746,   318,  2968,   306,  1900,   355,
           262,  1936,  9539,   286, 18522,    11,   996,   517, 14351,    11,
           262,  2746,  1281, 15968,   257, 17085,   286,  7016,  2585,  5924,
           416,  5651,   453,  2801,  3871,   706, 13669,    13,   383,  1936,
          9539,   389, 16199, 13437,    25, 14425,    11,  8993,    11, 23189,
            11,  8862,   290, 13427,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}